In [21]:
import ast
import configparser
import pandas as pd
import psycopg2

In [22]:
NOT_FOUND = []

In [23]:
config = configparser.ConfigParser()

In [24]:
config.read('config.ini')

['config.ini']

In [25]:
def open_connection():
    conn = \
        psycopg2.connect( \
            host = config['ro-prod']['host'],
            database = config['ro-prod']['name'],
            user = config['ro-prod']['user'],
            password = config['ro-prod']['pass'])
    
    conn.autocommit = True
    cur = conn.cursor()
    
    return conn, cur

In [26]:
CONN, CUR = open_connection()

In [27]:
def close_connection(conn, cur):
    cur.close()
    conn.close()

In [28]:
def get_product_name(barcode):
    barcode_iqvia = barcode.zfill(15)
    
    query = f" \
        SELECT IP.\"PRODUTO\" \
        FROM iqvia_produtos IP \
        WHERE IP.\"EAN\" = '{barcode_iqvia}';"            
    CUR.execute(query)
    res = CUR.fetchone()

    if res:
        return ''.join(res).strip()
    else:
        query = f" \
            SELECT DSP.\"name\" \
            FROM data_storeproduct DSP \
            WHERE DSP.barcode = '{barcode}' \
            ORDER BY DSP.last_change DESC;"            
        CUR.execute(query)
        res = CUR.fetchone()

        if res:
            return ''.join(res).strip()
        else:
            return barcode

In [29]:
def translate(list_):
    list_ = ast.literal_eval(list_)
    products = []
    
    for barcode in list_:
        name = get_product_name(barcode)
        products.append(name)

        if name.isnumeric():
            NOT_FOUND.append(name)
    
    return ', '.join(products)

In [30]:
path = 'outputs/by-store'

In [31]:
df = pd.read_csv(f"{path}/association_rules.csv")

In [32]:
df = df[df['lift'] >= 1].sort_values(by = 'lift', ascending = False)

In [33]:
df['antecedent'] = df['antecedent'].apply(lambda x: translate(x))

In [34]:
df['consequent'] = df['consequent'].apply(lambda x: translate(x))

In [35]:
close_connection(CONN, CUR)

In [36]:
df.to_csv(f"{path}/association_rules_translated.csv")

In [37]:
NOT_FOUND = list(set(NOT_FOUND))

In [38]:
len(NOT_FOUND)

32

In [39]:
df_not_found = pd.DataFrame(data = NOT_FOUND, columns = ['barcode']).reset_index(drop = True)

In [40]:
df_not_found.to_csv(f"{path}/not-found.csv", index = False)